In [1]:
spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

StatementMeta(, 4988eab5-09a2-459f-852d-1fcda3d3840b, 3, Finished, Available)

In [2]:
%%sql

CREATE OR REPLACE TABLE LH_AP_Gold.payment_terms
USING DELTA AS 
SELECT
	LH_AP_Silver.T052.ZTERM_Terms_of_payment_key AS terms_of_payment
	,CAST(LH_AP_Silver.T052.ZTAG1_Days_from_Baseline_Date_for_Payment AS INT) AS days_from_baseline_date_for_payment
	,CASE
		WHEN LH_AP_Silver.TVZBT.VTEXT_Description_of_terms_of_payment = '' OR LH_AP_Silver.TVZBT.VTEXT_Description_of_terms_of_payment IS NULL
		THEN 'No description for terms'
		ELSE LH_AP_Silver.TVZBT.VTEXT_Description_of_terms_of_payment
	END AS payment_terms_description
	,(LH_AP_Silver.T052.ZTERM_Terms_of_payment_key || ' - ' || payment_terms_description) AS payment_terms_combined_description
FROM LH_AP_Silver.T052
LEFT JOIN LH_AP_Silver.TVZBT
ON LH_AP_Silver.TVZBT.ZTERM_Terms_of_payment_key = LH_AP_Silver.T052.ZTERM_Terms_of_payment_key
WHERE LH_AP_Silver.TVZBT.SPRAS_Language_Key = 'E' OR LH_AP_Silver.TVZBT.SPRAS_Language_Key IS NULL

StatementMeta(, ae9833aa-29b0-40b2-976b-155c21350cee, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql

CREATE OR REPLACE TABLE LH_AP_Gold.document_types
USING DELTA AS
SELECT
    LH_AP_Silver.T003T.BLART_Document_type AS document_type
    ,CASE
		WHEN LH_AP_Silver.T003T.LTEXT_Document_Type_Description = '' OR LH_AP_Silver.T003T.LTEXT_Document_Type_Description IS NULL
		THEN 'No description for document type'
		ELSE LH_AP_Silver.T003T.LTEXT_Document_Type_Description
	END AS document_type_description
	,(LH_AP_Silver.T003T.BLART_Document_type || ' - ' || document_type_description) AS document_type_combined_description
FROM LH_AP_Silver.T003T
WHERE LH_AP_Silver.T003T.SPRAS_Language_Key = 'E'

StatementMeta(, ae9833aa-29b0-40b2-976b-155c21350cee, 5, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql

CREATE OR REPLACE TABLE LH_AP_Gold.company_codes
USING DELTA AS
SELECT
      LH_AP_Silver.T001.BUKRS_Company_Code AS company_code
      ,LH_AP_Silver.T001.BUTXT_Name_of_Company_Code_or_Company AS company_description
	  ,(LH_AP_Silver.T001.BUKRS_Company_Code || ' - ' || LH_AP_Silver.T001.BUTXT_Name_of_Company_Code_or_Company) AS company_code_combined_description
	  ,LH_AP_Silver.T001.KTOPL_Chart_of_Accounts AS chart_of_accounts
FROM LH_AP_Silver.T001
WHERE LH_AP_Silver.T001.KTOPL_Chart_of_Accounts = 'ZCOA'

StatementMeta(, ae9833aa-29b0-40b2-976b-155c21350cee, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql

CREATE OR REPLACE TABLE LH_AP_Gold.vendors
USING DELTA AS
SELECT
	LH_AP_Silver.LFA1.LIFNR_Account_Number_of_Vendor_or_Creditor AS vendor_number
	,CAST(CAST(LH_AP_Silver.LFA1.LIFNR_Account_Number_of_Vendor_or_Creditor AS INTEGER) AS VARCHAR(10)) AS vendor_number_trimmed
	,LH_AP_Silver.LFA1.NAME1_Name_1 AS vendor_name_1
	,LH_AP_Silver.LFA1.NAME2_Name_2 AS vendor_name_2
	,(CAST(CAST(LH_AP_Silver.LFA1.LIFNR_Account_Number_of_Vendor_or_Creditor AS INTEGER) AS VARCHAR(10)) || ' - ' || LH_AP_Silver.LFA1.NAME1_Name_1) AS vendor_combined_description
	,LH_AP_Silver.LFA1.SORTL_Sort_field AS sort_field
	,LH_AP_Silver.LFA1.PSTLZ_Postal_Code AS postal_code
	,LH_AP_Silver.LFA1.ORT01_City AS city
	,LH_AP_Silver.LFA1.REGIO_Region_State_Province_County AS region
	,LH_AP_Silver.LFA1.LAND1_Country_Key AS country_key
	,LH_AP_Silver.LFA1.STRAS_Street_and_House_Number AS address
	,LH_AP_Silver.LFA1.TELF1_First_telephone_number AS telephone_number
    ,LH_AP_Silver.LFA1.TELFX_Fax_Number AS fax_number
FROM 
	LH_AP_Silver.LFA1;

StatementMeta(, ae9833aa-29b0-40b2-976b-155c21350cee, 15, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

StatementMeta(, 77b5b5e6-128c-498b-be9f-f157416639a7, 15, Finished, Available)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'spark'.(line 1, pos 0)

== SQL ==
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
^^^
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")


In [6]:
%%sql
CREATE OR REPLACE TABLE LH_AP_Gold.ap_line_items
USING DELTA
AS
SELECT
    LH_AP_Silver.0FI_AP_4.BUKRS_Company_Code AS company_code,
    LH_AP_Silver.0FI_AP_4.FISCPER_Fiscal_year___period AS fiscal_period,
    LH_AP_Silver.0FI_AP_4.BELNR_Accounting_Document_Number AS accounting_document,
    LH_AP_Silver.0FI_AP_4.BUZEI_Number_of_Line_Item_Within_Accounting_Document AS accounting_document_ttem,
    LH_AP_Silver.0FI_AP_4.STATUSPS_Status_of_FI_Item AS status_of_item,
    LH_AP_Silver.0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor AS vendor_number,
    MajorBUT0IS.IND_SECTOR_Industry AS major_commodity_code,
    MinorBUT0IS.IND_SECTOR_Industry AS minor_commodity_code,
    LH_AP_Silver.LFA1.KTOKK_Supplier_Account_Group AS account_group,
    --LH_AP_Silver.LFB1.ZWELS_List_of_Respected_Payment_Methods AS payment_method,
    --LH_AP_Silver.LFM1.EKORG_Purchasing_organization AS purchasing_org,
    LH_AP_Silver.0FI_AP_4.BLART_Document_type AS document_type,
    LH_AP_Silver.0FI_AP_4.HKONT_General_Ledger_Account AS account,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE) AS document_date,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.BUDAT_Posting_Date_in_the_Document AS DATE) AS posting_date,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.CPUDT_Day_On_Which_Accounting_Document_Was_Entered AS DATE) AS entry_date,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE) AS clearing_date,
    LH_AP_Silver.0FI_AP_4.AUGBL_Document_Number_of_the_Clearing_Document AS clearing_document,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE) AS net_due_date,
    dayofweek(Net_Due_Date) AS due_day_of_week, -- Sunday = 1, Monday = 2, ..., Saturday = 7
    dayofmonth(Net_Due_Date) AS due_day_of_month,
    weekofyear(Net_Due_Date) AS due_week_of_year,
    dayofyear(Net_Due_Date) AS due_day_of_year,
    TRY_CAST(LH_AP_Silver.0FI_AP_4.ZFBDT_Baseline_Date_for_Due_Date_Calculation AS DATE) AS baseline_date,
    LH_AP_Silver.0FI_AP_4.LAND1_Country_Key AS country_key,
    LH_AP_Silver.0FI_AP_4.ZTERM_Terms_of_Payment_Key AS payment_terms,
    --LH_AP_Silver.LFB1.ZTERM_Terms_of_payment_key AS payment_terms_vendor_master,
    LH_AP_Silver.0FI_AP_4.ZLSPR_Payment_Block_Key AS payment_block_key,
    LH_AP_Silver.0FI_AP_4.LCURR_Currency_key_of_the_local_currency AS local_currency_key,
    CASE
        WHEN LH_AP_Silver.0FI_AP_4.SHKZG_Debit_Credit_Indicator = 'S' THEN (LH_AP_Silver.0FI_AP_4.DMBTR_Amount_in_local_currency * -1)
        ELSE LH_AP_Silver.0FI_AP_4.DMBTR_Amount_in_local_currency
    END AS amount_local_currency,
    CASE
        WHEN LH_AP_Silver.0FI_AP_4.SHKZG_Debit_Credit_Indicator = 'S' THEN (LH_AP_Silver.0FI_AP_4.DMBE2_Amount_in_Second_Local_Currency_or_Group_Currency * -1)
        ELSE LH_AP_Silver.0FI_AP_4.DMBE2_Amount_in_Second_Local_Currency_or_Group_Currency
    END AS amount_group_currency,
    LH_AP_Silver.0FI_AP_4.SHKZG_Debit_Credit_Indicator AS debit_credit_indicator,
    LH_AP_Silver.0FI_AP_4.SGTXT_Item_Text AS item_text,
    DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE)) AS delta_days_doc_clearing,
    DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE)) AS delta_days_doc_due,
    DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE)) AS delta_days_due_cleared,
    CASE 
        WHEN DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE)) < -2 THEN 'Early'
        WHEN DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE)) BETWEEN -2 AND 0 THEN 'On-time'
        WHEN DATEDIFF(day, TRY_CAST(LH_AP_Silver.0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(LH_AP_Silver.0FI_AP_4.AUGDT_Clearing_Date AS DATE)) > 0 THEN 'Late'
    END AS payment_status
FROM LH_AP_Silver.0FI_AP_4

-- Join commodity code table
LEFT JOIN LH_AP_Silver.BUT0IS AS MajorBUT0IS
ON MajorBUT0IS.PARTNER_Business_Partner_Number = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND TRIM(MajorBUT0IS.ISTYPE_Industry_System) = 'AGC'
AND MajorBUT0IS.ISDEF_Industry_is_Standard_for_BP_in_Industry_System = 'X'
LEFT JOIN LH_AP_Silver.BUT0IS AS MinorBUT0IS
ON MinorBUT0IS.PARTNER_Business_Partner_Number = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND TRIM(MinorBUT0IS.ISTYPE_Industry_System) = 'AGC'
AND MinorBUT0IS.ISDEF_Industry_is_Standard_for_BP_in_Industry_System = ''
-- Join vendor table
LEFT JOIN LH_AP_Silver.LFB1
ON LH_AP_Silver.LFB1.LIFNR_Account_Number_of_Vendor_or_Creditor = LH_AP_Silver.0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND LH_AP_Silver.0FI_AP_4.BUKRS_Company_Code = LH_AP_Silver.LFB1.BUKRS_Company_Code
-- Join CC vendor table
--LEFT JOIN LH_AP_Silver.LFM1
--ON LH_AP_Silver.LFM1.`LIFNR_Vendor's_account_number` = LH_AP_Silver.0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
LEFT JOIN LH_AP_Silver.LFA1
ON LH_AP_Silver.LFA1.LIFNR_Account_Number_of_Vendor_or_Creditor = LH_AP_Silver.0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor

WHERE
TRY_CAST(LH_AP_Silver.0FI_AP_4.ZFBDT_Baseline_Date_for_Due_Date_Calculation AS DATE) > '1582-10-15' 
AND TRY_CAST(LH_AP_Silver.0FI_AP_4.ZFBDT_Baseline_Date_for_Due_Date_Calculation AS DATE) < '2026-01-01'
AND TRY_CAST(LH_AP_Silver.0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE) > '1582-10-15' 
AND TRY_CAST(LH_AP_Silver.0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE) < '2026-01-01'
--AND BELNR_Accounting_Document_Number = '1900007532'

StatementMeta(, 59d428c8-bf74-4cdf-863b-0aaa78a88b32, 7, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [1]:
%%sql
SELECT *
FROM LH_AP_Silver.0FI_AP_4
WHERE BELNR_Accounting_Document_Number = '1900007532'

StatementMeta(, 3bad9a17-2a2a-4dab-a6bf-a48a99db6817, 2, Finished, Available)

<Spark SQL result set with 3 rows and 84 fields>

In [3]:
%%sql
CREATE OR REPLACE TABLE LH_AP_Gold.AP_Line_Items_Enhanced
USING DELTA
AS
SELECT 
    0FI_AP_4.BUKRS_Company_Code AS Company_Code,
    0FI_AP_4.FISCPER_Fiscal_year___period AS Fiscal_Period,
    0FI_AP_4.BELNR_Accounting_Document_Number AS Accounting_Document,
    0FI_AP_4.BUZEI_Number_of_Line_Item_Within_Accounting_Document AS Accounting_Document_Item,
    0FI_AP_4.STATUSPS_Status_of_FI_Item AS Status_of_Item,
    0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor AS Vendor_Number,
    MajorBUT0IS.IND_SECTOR_Industry AS Major_Commodity_Code,
    MinorBUT0IS.IND_SECTOR_Industry AS Minor_Commodity_Code,
    LFA1.KTOKK_Supplier_Account_Group AS Account_Group,
    LFB1.ZWELS_List_of_Respected_Payment_Methods AS Payment_Method,
    --LFM1.EKORG_Purchasing_organization AS Purchasing_Org,
    0FI_AP_4.BLART_Document_type AS Document_Type,
    0FI_AP_4.HKONT_General_Ledger_Account AS Recon_Account,
    TRY_CAST(0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE) AS Document_Date,
    TRY_CAST(0FI_AP_4.BUDAT_Posting_Date_in_the_Document AS DATE) AS Posting_Date,
    TRY_CAST(0FI_AP_4.CPUDT_Day_On_Which_Accounting_Document_Was_Entered AS DATE) AS Entry_Date,
    TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE) AS Clearing_Date,
    0FI_AP_4.AUGBL_Document_Number_of_the_Clearing_Document AS Clearing_Document,
    TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE) AS Net_Due_Date,
    dayofweek(Net_Due_Date) AS Due_Day_Of_Week, -- Sunday = 1, Monday = 2, ..., Saturday = 7
    dayofmonth(Net_Due_Date) AS Due_Day_Of_Month,
    weekofyear(Net_Due_Date) AS Due_Week_Of_Year,
    dayofyear(Net_Due_Date) AS Due_Day_Of_Year,
    TRY_CAST(0FI_AP_4.ZFBDT_Baseline_Date_for_Due_Date_Calculation AS DATE) AS Baseline_Date,
    0FI_AP_4.LAND1_Country_Key AS Country_Key,
    --0FI_AP_4.ZTERM_Terms_of_Payment_Key AS Payment_Terms_Key,
    LFB1.ZTERM_Terms_of_payment_key AS Payment_Terms_LFB1,
    0FI_AP_4.ZLSPR_Payment_Block_Key AS Payment_Block_Key,
    0FI_AP_4.LCURR_Currency_key_of_the_local_currency AS Local_Currency_Key,
    0FI_AP_4.HWAE2_Currency_Key_of_Second_Local_Currency AS Group_Currency_Key,
    CASE
        WHEN 0FI_AP_4.SHKZG_Debit_Credit_Indicator = 'S' THEN (0FI_AP_4.DMBTR_Amount_in_local_currency * -1)
        ELSE 0FI_AP_4.DMBTR_Amount_in_local_currency
    END AS Amount_Local_Currency,
    CASE
        WHEN 0FI_AP_4.SHKZG_Debit_Credit_Indicator = 'S' THEN (0FI_AP_4.DMBE2_Amount_in_Second_Local_Currency_or_Group_Currency * -1)
        ELSE 0FI_AP_4.DMBE2_Amount_in_Second_Local_Currency_or_Group_Currency
    END AS Amount_Group_Currency,
    0FI_AP_4.SHKZG_Debit_Credit_Indicator AS Debit_Credit_Indicator,
    0FI_AP_4.SGTXT_Item_Text AS Item_Text,
    DATEDIFF(day, TRY_CAST(0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE), TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE)) AS Delta_Days_Doc_Clearing,
    DATEDIFF(day, TRY_CAST(0FI_AP_4.BLDAT_Document_Date_in_Document AS DATE), TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE)) AS Delta_Days_Doc_Due,
    DATEDIFF(day, TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE)) AS Delta_Days_Due_Cleared,
    CASE 
        WHEN DATEDIFF(day, TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE)) < -2 THEN 'Early'
        WHEN DATEDIFF(day, TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE)) BETWEEN -2 AND 0 THEN 'On-time'
        WHEN DATEDIFF(day, TRY_CAST(0FI_AP_4.NETDT_Net_Due_Date AS DATE), TRY_CAST(0FI_AP_4.AUGDT_Clearing_Date AS DATE)) > 0 THEN 'Late'
    END AS Payment_Status
FROM 0FI_AP_4

-- Join commodity code table
LEFT JOIN BUT0IS AS MajorBUT0IS
ON MajorBUT0IS.PARTNER_Business_Partner_Number = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND TRIM(MajorBUT0IS.ISTYPE_Industry_System) = 'AGC'
AND MajorBUT0IS.ISDEF_Industry_is_Standard_for_BP_in_Industry_System = 'X'
LEFT JOIN BUT0IS AS MinorBUT0IS
ON MinorBUT0IS.PARTNER_Business_Partner_Number = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND TRIM(MinorBUT0IS.ISTYPE_Industry_System) = 'AGC'
AND MinorBUT0IS.ISDEF_Industry_is_Standard_for_BP_in_Industry_System = ''
-- Join vendor table

LEFT JOIN LFB1
ON LFB1.LIFNR_Account_Number_of_Vendor_or_Creditor = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
AND 0FI_AP_4.BUKRS_Company_Code = LFB1.BUKRS_Company_Code

-- Join CC vendor table
-- LEFT JOIN LFM1
-- ON LFM1.`LIFNR_Vendor's_account_number` = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor
LEFT JOIN LFA1
ON LFA1.LIFNR_Account_Number_of_Vendor_or_Creditor = 0FI_AP_4.LIFNR_Account_Number_of_Vendor_or_Creditor

WHERE 
-- Remove duplicate line for vendor payment (ZP)
0FI_AP_4.BLART_Document_type = 'KR'
-- Remove intercompany and employees
AND LFA1.KTOKK_Supplier_Account_Group NOT IN ('EMPV', 'LIEF')
;

StatementMeta(, 76e2fa44-7fb8-4774-8dd2-b10ecba7830c, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
%%sql
CREATE OR REPLACE TABLE LH_AP_Gold.12M_Vendor_Aggregated_Invoice_Metrics
    USING DELTA
    AS
    SELECT 
        Vendor_Number,
        COUNT(Accounting_Document) AS CNT_PAID,
        SUM(CASE WHEN Payment_Status = 'Late' THEN 1 ELSE 0 END) AS CNT_LATE,
        SUM(CASE WHEN Payment_Status = 'On-time' THEN 1 ELSE 0 END) AS CNT_ONTIME,
        SUM(CASE WHEN Payment_Status = 'Early' THEN 1 ELSE 0 END) AS CNT_EARLY,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Late' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_LATE,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'On-time' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_ONTIME,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Early' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_EARLY,
        SUM(Amount_Group_Currency) AS VAL_TOTAL,
        SUM(CASE WHEN Payment_Status = 'Late' THEN Amount_Group_Currency ELSE 0 END) AS VAL_LATE,
        SUM(CASE WHEN Payment_Status = 'On-time' THEN Amount_Group_Currency ELSE 0 END) AS VAL_ONTIME,
        SUM(CASE WHEN Payment_Status = 'Early' THEN Amount_Group_Currency ELSE 0 END) AS VAL_EARLY,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Late' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_LATE,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'On-time' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_ONTIME,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Early' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_EARLY,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'Late' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_LATE,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'On-time' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_ONTIME,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'Early' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_EARLY,
        ROUND(AVG(Amount_Group_Currency), 2) AS AVG_INV_VAL
    FROM 
        AP_Line_Items_Enhanced
    WHERE 
        LTRIM(RTRIM(Clearing_Document)) <> ''
        AND Posting_Date >= add_months(current_date(), -12)
    GROUP BY 
        Vendor_Number;


StatementMeta(, 76e2fa44-7fb8-4774-8dd2-b10ecba7830c, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
# List of months for which to generate the metrics
months_list = [3, 6, 12]

for months in months_list:
    # Dynamically create the table name based on the months
    table_name = f"{months}M_Vendor_Aggregated_Invoice_Metrics"
    
    # Construct the SQL query with the dynamic months and table name
    query = f"""
    CREATE OR REPLACE TABLE {table_name}
    USING DELTA
    AS
    SELECT 
        Vendor_Number,
        COUNT(Accounting_Document) AS CNT_PAID,
        SUM(CASE WHEN Payment_Status = 'Late' THEN 1 ELSE 0 END) AS CNT_LATE,
        SUM(CASE WHEN Payment_Status = 'On-time' THEN 1 ELSE 0 END) AS CNT_ONTIME,
        SUM(CASE WHEN Payment_Status = 'Early' THEN 1 ELSE 0 END) AS CNT_EARLY,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Late' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_LATE,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'On-time' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_ONTIME,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Early' THEN 1 ELSE 0 END) AS FLOAT) / COUNT(Accounting_Document), 3) AS RATIO_EARLY,
        SUM(Amount_Group_Currency) AS VAL_TOTAL,
        SUM(CASE WHEN Payment_Status = 'Late' THEN Amount_Group_Currency ELSE 0 END) AS VAL_LATE,
        SUM(CASE WHEN Payment_Status = 'On-time' THEN Amount_Group_Currency ELSE 0 END) AS VAL_ONTIME,
        SUM(CASE WHEN Payment_Status = 'Early' THEN Amount_Group_Currency ELSE 0 END) AS VAL_EARLY,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Late' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_LATE,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'On-time' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_ONTIME,
        ROUND(CAST(SUM(CASE WHEN Payment_Status = 'Early' THEN Amount_Group_Currency ELSE 0 END) AS FLOAT) / SUM(Amount_Group_Currency), 3) AS RATIO_VAL_EARLY,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'Late' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_LATE,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'On-time' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_ONTIME,
        ROUND(AVG(ABS(CASE WHEN Payment_Status = 'Early' THEN Delta_Days_Due_Cleared END)), 0) AS AVG_DAYS_EARLY,
        ROUND(AVG(Amount_Group_Currency), 2) AS AVG_INV_VAL
    FROM 
        AP_Line_Items_Enhanced
    WHERE 
        LTRIM(RTRIM(Clearing_Document)) <> ''
        AND Posting_Date >= add_months(current_date(), -{months})
    GROUP BY 
        Vendor_Number;
    """
    
    # Execute the SQL query
    spark.sql(query)

In [ ]:
%%sql
CREATE OR REPLACE TABLE LH_AP_Gold.ap_line_items_scored
USING DELTA
AS
SELECT 
	LH_AP_Gold.ap_line_items.company_code
	,LH_AP_Gold.ap_line_items.fiscal_period
	,LH_AP_Gold.ap_line_items.accounting_document
	,LH_AP_Gold.ap_line_items.accounting_document_ttem as accounting_document_item
	,LH_AP_Gold.ap_line_items.status_of_item
	,LH_AP_Gold.ap_line_items.vendor_number
	,LH_AP_Gold.ap_line_items.major_commodity_code
	,LH_AP_Gold.ap_line_items.minor_commodity_code
	,LH_AP_Gold.ap_line_items.account_group
	,LH_AP_Gold.ap_line_items.document_type
	,LH_AP_Gold.ap_line_items.account
	,LH_AP_Gold.ap_line_items.document_date
	,LH_AP_Gold.ap_line_items.posting_date
	,LH_AP_Gold.ap_line_items.entry_date
	,LH_AP_Gold.ap_line_items.clearing_date
	,LH_AP_Gold.ap_line_items.clearing_document
	,LH_AP_Gold.ap_line_items.net_due_date
	,LH_AP_Gold.ap_line_items.due_day_of_week
	,LH_AP_Gold.ap_line_items.due_day_of_month
	,LH_AP_Gold.ap_line_items.due_week_of_year
	,LH_AP_Gold.ap_line_items.due_day_of_year
	,LH_AP_Gold.ap_line_items.baseline_date
	,LH_AP_Gold.ap_line_items.country_key
	,LH_AP_Gold.ap_line_items.payment_terms
	,LH_AP_Gold.ap_line_items.payment_block_key
	,LH_AP_Gold.ap_line_items.local_currency_key
	,LH_AP_Gold.ap_line_items.amount_local_currency
	,LH_AP_Gold.ap_line_items.amount_group_currency
	,LH_AP_Gold.ap_line_items.debit_credit_indicator
	,LH_AP_Gold.ap_line_items.item_text
	,LH_AP_Gold.ap_line_items.delta_days_doc_clearing
	,LH_AP_Gold.ap_line_items.delta_days_doc_due
	,LH_AP_Gold.ap_line_items.delta_days_due_cleared
	,LH_AP_Gold.ap_line_items.payment_status
	,LH_AP_Gold.ap_line_items_predictions.predictions as predicted_days_to_pay
	,LH_AP_Gold.ap_line_items_predictions.Predicted_Status as predicted_status
FROM LH_AP_Gold.ap_line_items
LEFT JOIN LH_AP_Gold.ap_line_items_predictions
	ON LH_AP_Gold.ap_line_items.company_code = LH_AP_Gold.ap_line_items_predictions.Company_Code
    AND LH_AP_Gold.ap_line_items.fiscal_period = LH_AP_Gold.ap_line_items_predictions.Fiscal_Period
    AND LH_AP_Gold.ap_line_items.accounting_document = LH_AP_Gold.ap_line_items_predictions.Accounting_Document
    AND LH_AP_Gold.ap_line_items.accounting_document_ttem = LH_AP_Gold.ap_line_items_predictions.Accounting_Document_Item

StatementMeta(, eed05cca-095a-4a8d-915e-e26e5629e15b, 6, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>